In [1]:
import pandas as pd
import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
list_cluster= glob.glob('/content/drive/MyDrive/Colab Notebooks/km25/*clean*')

In [ ]:
print(list_cluster)

In [29]:
def read_cluster(files):
  liste=[]
  for i,cluster in enumerate(files):
    with open(cluster, "r") as file:
      liste.append(file.readlines())
  return liste

In [30]:
dic= read_cluster(list_cluster)
print(len(dic))

25


In [31]:
print(len(dic[24]))

20147


In [8]:
for i in range(len(list_cluster)):
  vars()["data_"+str(i)]=pd.DataFrame(dic[i], columns = ['tweet'])

In [19]:
print(data_10.head(20))

                                                tweet
0   ethusd dly doublement cadrer ich ratio ext fib...
1               policier convaincre lfi côté plaque\n
2                                       heure crime\n
3   mars baye cheikh diop âgé an tuer balle garage...
4   mars mansour thiam tuer balle colobane luire v...
5                                  fautes ca france\n
6   vidéo élection voler dominion faveur macron pr...
7   autorité moral financement étranger national a...
8                                         éducation\n
9                                          allemand\n
10                                   final européen\n
11                   lien corruption politique mdrr\n
12  minute temps accorder écologie débat primaire ...
13  respect parti politique geopo droit bien viser...
14  euro primeinflation pauvre décider sénateur pr...
15                   euro prime noël sénateur voter\n
16                     euro étudiant macron arriver\n
17  euro amende requérir eri

In [ ]:
!pip install gensim==4.0
!pip install python-Levenshtein

In [20]:
from gensim.models import KeyedVectors

In [21]:
w2v = KeyedVectors.load('/content/drive/MyDrive/Colab Notebooks/w2vec_model_d300_1M')

In [22]:
import numpy as np

In [23]:
#encode un message
def encode(msg):
    return np.mean([w2v[word] for word in msg if word in w2v] or [np.zeros(300)], axis = 0)

In [ ]:
w2v.similar_by_word("sale")

[('sal', 0.6782066822052002),
 ('pvte', 0.629138708114624),
 ('mèrde', 0.6162537336349487),
 ('pute', 0.6096962094306946),
 ('pte', 0.6044666767120361),
 ('clochard', 0.6014346480369568),
 ('slp', 0.6004031896591187),
 ('crasseux', 0.5957321524620056),
 ('puent', 0.5908592939376831),
 ('daronn', 0.5900101661682129)]

In [32]:
def encoded(liste):
  w2v=[]
  for i in range(len(liste)):
    w2v.append(np.array([encode(msg.split()) for msg in liste[i]]))
  return w2v

In [33]:
%%time
#encodage de l'échantillon pour créer le jeu de données
dataset_encoded = encoded(dic)

CPU times: user 32.4 s, sys: 993 ms, total: 33.4 s
Wall time: 32.8 s


In [35]:
print(len(dataset_encoded[24]))

20147


In [ ]:
#sauvegarder mes embeddings
%%time
"""for i in range(len(dataset_encoded)):
  with open(f"w2v/w2v{i}.txt", 'a') as f:
      for row in dataset_encoded[i]:
        #np.savetxt(f, row)"""

In [36]:
from sklearn.ensemble import IsolationForest

clf=IsolationForest(n_estimators=100, max_samples='auto', \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)


In [37]:
%%time
for i in range(len(dataset_encoded)):
  clf.fit(dataset_encoded[i])

CPU times: user 2min 44s, sys: 1.67 s, total: 2min 46s
Wall time: 1min 27s


In [39]:
%%time
pred=[]
for i in range(len(dataset_encoded)):
  pred.append(clf.predict(dataset_encoded[i]))

CPU times: user 1min 5s, sys: 568 ms, total: 1min 6s
Wall time: 1min 6s


In [40]:
print(pred[1])

[ 1 -1  1 ... -1  1  1]


In [41]:
for i in range(len(dataset_encoded)):
  vars()["data_"+str(i)]['pred']= pred[i]

In [42]:
print(data_1.head(20))

                                                tweet  pred
0                                            voi lt\n     1
1                         bel douce nuit louaner cc\n    -1
2                            lune incroyablemer bel\n     1
3   excellent technique animalcrossing acnh ninten...    -1
4                                         excellent\n    -1
5                                     chance calmer\n     1
6                                     lt ilyyyysmmm\n    -1
7           anniversaire recevoir nude anniversaire\n    -1
8                                       gentil jour\n    -1
9                               soirée dokkanbattle\n     1
10                                  ange gardien lt\n     1
11              an alhamdoulila joyeux anniversaire\n    -1
12                                        chance fé\n     1
13                                        chance fé\n     1
14                                        chance fé\n     1
15                                      

In [44]:
# Trouver les outliers de chaque cluster
# ceux dont l'index est -1
outliers=[]
for i in range(len(dataset_encoded)):
  outliers.append((vars()["data_"+str(i)].loc[vars()["data_"+str(i)]['pred']==-1]))
  outlier_index=list(outliers[i].index)
  print("cluster"+str(i)+":"+"\n")
  print(vars()["data_"+str(i)]['pred'].value_counts())
  print("\n")

cluster0:

-1    3150
 1      37
Name: pred, dtype: int64


cluster1:

-1    6915
 1    6045
Name: pred, dtype: int64


cluster2:

-1    1453
 1     217
Name: pred, dtype: int64


cluster3:

-1    3691
 1      68
Name: pred, dtype: int64


cluster4:

1    26684
Name: pred, dtype: int64


cluster5:

 1    24441
-1     8845
Name: pred, dtype: int64


cluster6:

 1    36494
-1     2093
Name: pred, dtype: int64


cluster7:

 1    89146
-1     7568
Name: pred, dtype: int64


cluster8:

 1    19406
-1     3228
Name: pred, dtype: int64


cluster9:

-1    1361
 1     740
Name: pred, dtype: int64


cluster10:

 1    45714
-1     3945
Name: pred, dtype: int64


cluster11:

 1    7539
-1    3623
Name: pred, dtype: int64


cluster12:

 1    55396
-1     4892
Name: pred, dtype: int64


cluster13:

 1    43742
-1     6140
Name: pred, dtype: int64


cluster14:

 1    59078
-1     7125
Name: pred, dtype: int64


cluster15:

 1    1759
-1    1206
Name: pred, dtype: int64


cluster16:

-1    9182
 1    

In [46]:
for i in range(len(outliers)):
  with open(f"drive/MyDrive/outliers25/outliers{i}", 'w') as f:
    #dfAsString = outliers[i].to_string((justify='justify'))
    dfAsString = str(outliers[i]['tweet'])
    f.write(dfAsString)